<a href="https://colab.research.google.com/github/FabianaFerrente/ibis_pipeline/blob/master/dest_ibis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')


shared_folder_path = '/content/drive/MyDrive/dest_IBIS'

# List files in the shared folder
if os.path.exists(shared_folder_path):
    files = os.listdir(shared_folder_path)
    print(f"Files in '{shared_folder_path}':")
    for file in files:
        print(file)
else:
    print(f"Folder '{shared_folder_path}' not found.")